Imports

In [1]:
import kblab, json
import os
import multiprocessing
import requests
import urllib3
import pandas as pd
from pandas.core.frame import DataFrame
from requests.auth import HTTPBasicAuth
from kblab import Archive
from kblab.utils import flerge
from json import loads
from tqdm import tqdm
from urllib3.util import Retry
from urllib3 import PoolManager, make_headers
from itertools import product
kblab.VERIFY_CA=False

In [ ]:
lorem = "scelerisque fermentum dui faucibus in ornare quam viverra orci sagittis eu volutpat odio facilisis mauris sit amet massa vitae tortor condimentum lacinia quis vel eros donec ac odio tempor orci dapibus ultrices in iaculis nunc sed augue lacus viverra vitae congue eu consequat ac felis donec et odio pellentesque diam volutpat commodo sed egestas egestas fringilla phasellus faucibus scelerisque eleifend donec pretium vulputate sapien nec sagittis aliquam malesuada bibendum arcu vitae elementum curabitur vitae nunc sed velit dignissim sodales ut eu sem integer vitae justo eget magna fermentum iaculis eu non diam phasellus vestibulum lorem sed risus ultricies tristique nulla aliquet enim tortor at auctor urna nunc id cursus metus aliquam eleifend mi in nulla posuere sollicitudin aliquam ultrices sagittis orci a scelerisque purus semper eget duis at tellus at urna condimentum mattis pellentesque id nibh tortor id aliquet lectus proin nibh nisl condimentum id venenatis a condimentum vitae sapien pellentesque habitant morbi tristique senectus et netus et malesuada fames ac turpis egestas sed tempus urna et pharetra pharetra massa massa ultricies mi quis hendrerit dolor magna eget est lorem ipsum dolor sit amet consectetur adipiscing elit pellentesque habitant morbi tristique senectus et netus et malesuada fames ac turpis egestas integer eget aliquet nibh praesent tristique magna sit amet purus gravida quis blandit turpis cursus in hac habitasse platea dictumst quisque sagittis purus sit amet volutpat consequat mauris nunc congue nisi vitae suscipit tellus mauris a diam maecenas sed enim ut sem viverra aliquet eget sit amet tellus cras adipiscing enim eu turpis egestas pretium aenean pharetra magna ac placerat vestibulum lectus mauris ultrices eros in cursus turpis massa tincidunt dui ut ornare lectus sit amet est placerat in egestas erat imperdiet sed euismod nisi porta lorem mollis aliquam ut porttitor leo a diam sollicitudin tempor id eu nisl nunc mi ipsum faucibus vitae aliquet nec ullamcorper sit amet risus nullam eget felis eget nunc lobortis mattis aliquam faucibus purus in massa tempor nec feugiat nisl pretium fusce id velit ut tortor pretium viverra suspendisse potenti nullam ac tortor vitae purus faucibus ornare suspendisse sed nisi lacus sed viverra tellus in hac habitasse platea dictumst vestibulum rhoncus est pellentesque elit ullamcorper dignissim cras tincidunt lobortis feugiat vivamus at augue eget arcu dictum varius duis at consectetur lorem donec massa sapien faucibus et molestie ac feugiat sed lectus vestibulum mattis ullamcorper velit sed ullamcorper morbi tincidunt ornare massa eget egestas purus viverra accumsan in nisl nisi scelerisque eu ultrices vitae auctor eu augue ut lectus arcu bibendum at varius vel pharetra vel turpis nunc eget lorem dolor sed viverra ipsum nunc aliquet bibendum enim facilisis gravida neque convallis a cras semper auctor neque vitae tempus quam pellentesque nec nam aliquam sem et tortor consequat id porta nibh venenatis cras sed felis eget velit aliquet sagittis id consectetur purus ut faucibus pulvinar elementum integer enim neque volutpat ac tincidunt vitae semper quis lectus nulla at volutpat diam ut venenatis tellus in metus vulputate eu scelerisque felis imperdiet proin fermentum leo vel orci porta non pulvinar neque laoreet suspendisse interdum consectetur libero id faucibus nisl"

In [10]:
with open('api_cred', 'r') as file:
    pw = file.read().replace('\n', '')

archive = Archive("https://betalab.kb.se", auth=("demo", pw))


In [11]:
def get_metadata(dark_id, headers):
  http = PoolManager(cert_reqs='CERT_NONE')
  kblab.VERIFY_CA=False
  try:
      print(f"trying https://betalab.kb.se/{dark_id}/meta.json")
      meta_json = http.request(
          "GET",
          f"https://betalab.kb.se/{dark_id}/meta.json",
          headers=headers,
          retries=Retry(connect=5, read=4, redirect=5, backoff_factor=0.02),
      )

      meta_json = loads(meta_json.data.decode("utf-8"))
      meta_json["dark_id"] = dark_id
      return meta_json

  except:
      print("failed getting")
      return {
          "dark_id": dark_id,
          "title": "failed",
          "year": "failed",
        }

In [12]:
archive.search('tags: "issue"')

meta_json_list = []
content_json_list = []
for package_id in archive.search('tags: "issue"', max=2):
    package = archive.get(package_id)
    lmao = flerge(package)

    if "meta.json" in package:
        meta_json = json.load(package.get_raw("meta.json"))
        meta_json_list.append(meta_json)
    if "content.json" in package:
        content_json = json.load(package.get_raw("content.json"))
        content_json_list.append(content_json)




In [ ]:
content_json_list = content_json_list[0]
lmao = [x["content"] for x in content_json_list]
lmao

In [14]:
content = [x[0]["content"] for x in content_json_list] #content_json_list[0][0]["content"]
content

['«•» Hemb&rnin ', 'Daglig morgontidning ^']

In [15]:

meta_json_list

[{'created': '1905-01-05',
  'year': '1905',
  'edition': '0',
  'issue': '12474B',
  'title': 'DAGENS NYHETER  1905-01-05'},
 {'created': '1905-02-17',
  'year': '1905',
  'edition': '0',
  'issue': '12516A',
  'title': 'DAGENS NYHETER  1905-02-17'}]

In [27]:
from sentence_transformers import SentenceTransformer

sentences = ["Han förtärde en närande och nyttig måltid.Han förtärde en närande och nyttig måltid.Han förtärde en närande och nyttig måltid.Han förtärde en närande och nyttig måltid.", 
"Varje exempel blir konverteradHan förtärde en närande och nyttig måltid.Han förtärde en närande och nyttig måltid.Han förtärde en närande och nyttig måltid.",
"Det var ett sunkigt hak med ganska gott käk.Han förtärde en närande och nyttig måltid.Han förtärde en närande och nyttig måltid.Han förtärde en närande och nyttig måltid.",
"Han inmundigade middagen tillsammans med ett glas rödvin.Han förtärde en närande och nyttig måltid.Han förtärde en närande och nyttig måltid.Han förtärde en närande och nyttig måltid.",
"Potatischips är jättegoda.Han förtärde en närande och nyttig måltid.Han förtärde en närande och nyttig måltid.Han förtärde en närande och nyttig måltid.",
"Tryck på knappen för att få tala med kundsupporten.Han förtärde en närande och nyttig måltid.Han förtärde en närande och nyttig måltid.Han förtärde en närande och nyttig måltid."]

source = "Mannen åt mat."

model = SentenceTransformer('KBLab/sentence-bert-swedish-cased')
embeddings = model.encode(sentences)
sourceEmb = model.encode(source)
print(embeddings)
print(sourceEmb)

[[ 0.08565636  0.11254321 -0.02355101 ...  0.00929855  0.05028643
  -0.12768164]
 [ 0.07838852  0.02995606 -0.03571041 ... -0.01521753  0.06298143
  -0.08363984]
 [ 0.04481409  0.07148021 -0.0313642  ...  0.00074999  0.0748238
  -0.0769266 ]
 [ 0.07923093  0.09726818 -0.04104416 ...  0.01661729  0.07949235
  -0.15745626]
 [-0.03911571  0.08857869 -0.02402698 ...  0.06262439  0.09870593
  -0.11599754]
 [ 0.09784506  0.05300535 -0.03907291 ...  0.06184708 -0.00861086
  -0.10053059]]
[ 1.32508457e-01  5.96463680e-03  3.43883149e-02  1.57257505e-02
 -1.55537473e-02 -5.31849032e-03 -2.18539521e-01  1.24811687e-01
  8.16078633e-02  7.94360694e-03  5.81024028e-02  1.13142831e-02
  8.09717998e-02 -4.54286374e-02  1.29294381e-01 -1.16230249e-02
  9.54005122e-02  9.73669887e-02 -2.11207625e-02  1.14677541e-01
 -1.22725256e-01 -1.85505915e-02  5.95387109e-02  1.82689447e-02
 -1.22966431e-02  9.88732278e-02 -3.05919331e-02  5.75979203e-02
 -5.62502556e-02 -1.23067714e-01 -1.32290602e-01 -1.9582817

In [24]:
len(embeddings[0])

768

In [15]:
len(sentences)

6

In [9]:
from sentence_transformers.util import semantic_search

hits = semantic_search(sourceEmb, embeddings)

In [10]:
hits

[[{'corpus_id': 0, 'score': 0.7916003465652466},
  {'corpus_id': 3, 'score': 0.5491536855697632},
  {'corpus_id': 2, 'score': 0.323931485414505},
  {'corpus_id': 4, 'score': 0.27011963725090027},
  {'corpus_id': 5, 'score': 0.007242000196129084},
  {'corpus_id': 1, 'score': -0.058579858392477036}]]

In [22]:
from sklearn.datasets import fetch_20newsgroups

docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))['data']


KeyboardInterrupt: 

In [23]:
from bertopic import BERTopic
topic_model = BERTopic(embedding_model=model)

topics, probs = topic_model.fit_transform(sentences)

KeyboardInterrupt: 